In [ ]:
#!/usr/bin/python3
import requests
from bs4 import BeautifulSoup
import re
import os
from time import sleep
import json
import sqlite3
import random

In [ ]:
# Supportive Functions
def load_web(link):
    """
    General: Load the webpage from a link. Return a soup project
    """
    
    headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml; "
              "q=0.9,image/webp,*/*;q=0.8",
    "Accept-Encoding": "text/html",
    "Accept-Language": "en-US,en;q=0.8,zh-CN;q=0.6,zh;q=0.4,zh-TW;q=0.2",
    "Content-Type": "application/x-www-form-urlencoded",
    "User-Agent": "Mozilla/5.0 (X11; Linux i686) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/32.0.1700.77 Safari/537.36"
    }

    keepit = 0
    while keepit == 0:
        try:
            page = requests.get(link, headers=headers, timeout=20)
            soup = BeautifulSoup(page.text, "lxml")
            keepit = 1
        except requests.exceptions.Timeout:
            print('    [Error] Timeout! Retrying...')
            print('    ' + link)

    return soup

In [ ]:
# Function to load the article
def extract_link(page):
    """
    To download the earning transcripts from seeking alpha
    """
    print('Analysing page ' + str(page))
    link = 'http://seekingalpha.com/earnings/earnings-call-transcripts/' + str(page)
    dat = load_web(link)

    arts = dat.find_all('a', class_ = 'dashboard-article-link')
    if re.search(r'\(.*?\)', x.text):
        ticker = re.search(r'\(\w+\)', x.text).group(0)
    else:
        ticker = ''
    arts = [{'link':'http://seekingalpha.com' + x['href'] + '?part=single',
             'title':x.text,
             'ticker':ticker,
             'file':0} for x in arts]
    
    return(arts)
    
    
def extract_art(i):
    """
    Extract the transcripts from SA
    Need the macro variable DB
    """
    dat2 = load_web(i['link'])
    try:
        article = dat2.find('div', class_ = 'sa-art').text
    except AttributeError:
        article = ''
        
    return article

In [ ]:
## Parsing Links
def collect_links(DB):
    """
    A wrapper function to collect the links
    """
    i = 1
    while i <= 4151:
        try:
            newLink = extract_link(i)
            with open(DB, 'rt') as f:
                oldLink = json.load(f)
            with open(DB, 'wt') as f:
                json.dump(oldLink + newLink, f)
            i = i + 1
            sleep(random.randint(2, 5))
        except requests.exceptions.ConnectionError:
            sleep(60)
        except requests.exceptions.ChunkedEncodingError:
            sleep(60)

In [ ]:
## Downloading module
def download_trans(DB):
    """
    A wrapper function to download the transcripts
    """
    # setup
    print('Setting up environments...')
    with open(DB, 'rt') as f:
        db = json.load(f)

    old = [x for x in db if x['file'] != 0]
    todown = [x for x in db if x['file'] == 0]
    id = max([x['file'] for x in db]) + 1
        
    # downloader
    i = 0
    batch = 0
    print('Downloading ' + str(len(todown)) + ' files...')
    while i < len(todown):
        print('    ' + str(i + 1) + ' file')
        try:
            paper = extract_art(todown[i])
            if paper != '':
                with open('trs/' + str(id) + '.txt', 'wt') as f:
                    f.write(paper)
                todown[i]['file'] = id
            else:
                todown[i]['file'] = -1

            id = id + 1
            i = i + 1
            batch = batch + 1
            
            if batch >= 20:         
                with open(DB, 'wt') as f:
                    json.dump(old + todown, f)
                print('    Updated the database')
                batch = 0
            
            sleep(random.randint(1, 2))
            
        except requests.exceptions.ConnectionError:
            sleep(60)
        except requests.exceptions.ChunkedEncodingError:
            sleep(60)

In [ ]:
# Main
DB = 'db.json'

# collect_links(DB)
download_trans(DB)